## Import balíčků

In [140]:
import pandas as pd
import numpy as np

## Načtení souborů

In [141]:
df_file_all = pd.read_csv('data/dc_file_modified2.csv')
df_evt_all = pd.read_csv('data/dc_evt_modified2.csv')

/tmp/ipykernel_25975/2963581421.py:1: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_file_all = pd.read_csv('data/dc_file_modified2.csv')
/tmp/ipykernel_25975/2963581421.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_evt_all = pd.read_csv('data/dc_evt_modified2.csv')


## Promazání zbytečných příznaků

příznaky jsme po jednom prošli, většinou měli příliš mnoho hodnot, byly duplicitní nebo nebyly podstatné pro naši analýzu.

In [142]:
df_file = df_file_all.drop(columns=['Unnamed: 0', 'id', 'filename', 'extra', 'datetime', 'file_path', 'name', 'typef', 'dir_type', 'file_type', 'is_allocated', 'is_allocated0', 'sha_256', 'epochtime', 'hour', 'minute'])
df_evt = df_evt_all.drop(columns=['filename_type', 'keywords'])

## Přetypování příznaků

In [143]:
bools = [
         'M', 'A', 'C', 'B',
         'file_stat', 'NTFS_file_stat', 'file_entry_shell_item', 'NTFS_USN_change',
         'filef', 'directory', 'link',
         'dir_appdata', 'dir_win', 'dir_user', 'dir_other',
         'file_executable', 'file_graphic', 'file_documents', 'file_ps', 'file_other',
         'mft', 'lnk_shell_items', 'olecf_olecf_automatic_destinations/lnk/shell_items', 'winreg_bagmru/shell_items', 'usnjrnl',
         'is_allocated1'
         ]

df_file['timestamp'] = df_file['timestamp'].astype('datetime64[ns]')

df_file[bools] = df_file[bools].astype('boolean')

# procento aktivit v mesici, kdy doslo k inidentu
# 100 * (df_file['timestamp'] >= '2020-09-01').sum() / df_file.shape[0]
dict(df_file.dtypes)

{'inode': dtype('int64'),
 'M': BooleanDtype,
 'A': BooleanDtype,
 'C': BooleanDtype,
 'B': BooleanDtype,
 'file_stat': BooleanDtype,
 'NTFS_file_stat': BooleanDtype,
 'file_entry_shell_item': BooleanDtype,
 'NTFS_USN_change': BooleanDtype,
 'filef': BooleanDtype,
 'directory': BooleanDtype,
 'link': BooleanDtype,
 'dir_appdata': BooleanDtype,
 'dir_win': BooleanDtype,
 'dir_user': BooleanDtype,
 'dir_other': BooleanDtype,
 'file_executable': BooleanDtype,
 'file_graphic': BooleanDtype,
 'file_documents': BooleanDtype,
 'file_ps': BooleanDtype,
 'file_other': BooleanDtype,
 'mft': BooleanDtype,
 'lnk_shell_items': BooleanDtype,
 'olecf_olecf_automatic_destinations/lnk/shell_items': BooleanDtype,
 'winreg_bagmru/shell_items': BooleanDtype,
 'usnjrnl': BooleanDtype,
 'is_allocated1': BooleanDtype,
 'file_size': dtype('int64'),
 'timestamp': dtype('<M8[ns]')}

### Příznak file_size

změníme file_size podle toho, do jakého intervalu spadá
```
0: 0
1: 1 - 1_000
2: 1_001 - 10_000
3: 10_001 - 100_000
4: 100_001 - 1_000_000
5: 1_000_001+
```

In [144]:
df_file['file_size+1_log'] = np.log(df_file['file_size'] + 1)

size_stamps = [-float('inf'), 0, 1_000, 10_000, 100_000, 1_000_000, float('inf')]
size_labels = ['0', '1-1_000', '1_001-10_000', '10_001-100_000', '100_001-1_000_000', '1_000_000+']
# make data ordinal based on size

df_file['file_size'] = pd.cut(df_file['file_size'], size_stamps, labels=False)
df_file['file_size'].value_counts()

0    572239
2     85508
1     82970
3     66466
4     30209
5      6471
Name: file_size, dtype: int64

In [145]:
df_file.sample(10)

,inode,M,A,C,B,file_stat,NTFS_file_stat,file_entry_shell_item,NTFS_USN_change,filef,...,file_other,mft,lnk_shell_items,olecf_olecf_automatic_destinations/lnk/shell_items,winreg_bagmru/shell_items,usnjrnl,is_allocated1,file_size,timestamp,file_size+1_log
674649,0,False,False,True,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2020-09-17 16:49:54,0.000000
278857,0,False,False,False,True,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2014-03-21 18:07:48,0.000000
350194,0,False,True,False,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2014-03-21 19:28:14,0.000000
772848,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-17 17:01:48,0.000000
582612,0,False,False,True,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2020-09-17 16:49:25,0.000000
793409,84656,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,True,False,0,2020-09-17 17:05:01,0.000000
117857,77368,False,True,False,True,True,False,False,False,True,...,True,False,False,False,False,False,True,3,2013-08-22 12:37:06,10.091998
278540,0,True,True,False,False,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2014-03-21 18:07:47,0.000000
80608,0,False,True,False,True,False,True,False,False,False,...,True,True,False,False,False,False,False,0,2013-08-22 06:59:23,0.000000
359418,43403,False,True,False,False,True,False,False,False,True,...,True,False,False,False,False,False,True,3,2014-03-21 19:46:38,11.029164


In [146]:
#? tohle by bylo nice, ale je to moc pomale
# def find_minute_frequency(row):
#     return df_file[(df_file['timestamp'] >= row['timestamp'] - pd.Timedelta(minutes=1)) \
                #    & (df_file['timestamp'] < row['timestamp'] + pd.Timedelta(minutes=1))].shape[0]

# df_file.apply(find_minute_frequency, axis=1)

In [147]:
floored_to_min = df_file['timestamp'].dt.floor('min')
counts = floored_to_min.value_counts().to_dict()

# minute_activity = df_file['timestamp'].apply(lambda x: counts[x.floor('min')])
# minute_activity.to_csv('data/minute_activity.csv')
minute_activity = pd.read_csv('data/minute_activity.csv', index_col=0)['timestamp']

In [148]:
df_file['minute_activity'] = minute_activity

In [155]:
inode_counts = df_file['inode'].value_counts().to_dict()

inode_activity = df_file['inode'].apply(lambda x: inode_counts[x])

In [156]:
df_file['inode_activity'] = inode_activity
df_file.drop(columns=['inode'], inplace=True)

In [158]:
df_file.sample(10)

,M,A,C,B,file_stat,NTFS_file_stat,file_entry_shell_item,NTFS_USN_change,filef,directory,...,lnk_shell_items,olecf_olecf_automatic_destinations/lnk/shell_items,winreg_bagmru/shell_items,usnjrnl,is_allocated1,file_size,timestamp,file_size+1_log,minute_activity,inode_activity
419433,True,True,True,True,False,True,False,False,False,False,...,False,False,False,False,False,0,2020-09-17 16:47:57,0.000000,39621,443141
65432,False,True,False,True,True,False,False,False,True,False,...,False,False,False,False,True,2,2013-08-22 06:53:03,7.363914,832,3
428429,True,True,True,True,False,True,False,False,False,False,...,False,False,False,False,False,0,2020-09-17 16:47:59,0.000000,39621,443141
199808,True,True,False,False,True,False,False,False,True,False,...,False,False,False,False,True,1,2013-08-22 15:26:21,5.993961,8210,3
343737,True,True,False,False,False,True,False,False,False,False,...,False,False,False,False,False,0,2014-03-21 19:02:52,0.000000,282,443141
330585,True,True,False,True,False,True,False,False,False,False,...,False,False,False,False,False,0,2014-03-21 18:49:00,0.000000,7752,443141
765754,False,False,True,False,False,False,False,True,False,False,...,False,False,False,True,False,0,2020-09-17 16:52:16,0.000000,5323,82086
660903,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,0,2020-09-17 16:49:35,0.000000,155184,443141
663016,True,True,True,True,False,True,False,False,False,False,...,False,False,False,False,False,0,2020-09-17 16:49:35,0.000000,155184,443141
468708,False,False,True,False,False,True,False,False,False,False,...,False,False,False,False,False,0,2020-09-17 16:48:35,0.000000,88515,443141
